In [1]:
import yaml
import torch
import torch.nn as nn
import numpy as np
import tqdm
import os
import argparse
from torch.backends import cudnn

cudnn.benchmark = True

def delete_module(name):
    import sys
    del sys.modules[name]

In [2]:
# delete_module('scorenet')
# from scorenet import Scorenet

In [3]:
# delete_module('critic')
# from critic import Criticnet

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
config_file = './test_config.yaml'

def dict2namespace(config):
    namespace = argparse.Namespace()
    for key, value in config.items():
        if isinstance(value, dict):
            new_value = dict2namespace(value)
        else:
            new_value = value
        setattr(namespace, key, new_value)
    return namespace

with open(config_file, 'r') as f:
    config = yaml.load(f)
        
cfg = dict2namespace(config)
cfg

/pkgs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


Namespace(data=Namespace(batch_size=32, cates=['airplane'], data_dir='data/ShapeNetCore.v2.PC15k', dataset_scale=1, dataset_type='shapenet15k', normalize_per_shape=False, normalize_std_per_axis=False, num_workers=4, recenter_per_shape=True, te_max_sample_points=2048, tr_max_sample_points=2048, type='datasets.pointflow_datasets'), inference=Namespace(num_points=2048, num_steps=10, step_size_ratio=1, weight=1), models=Namespace(scorenet=Namespace(dim=3, hidden_size=256, n_blocks=24, out_dim=3, param_likelihood=False, sigma_condition=True, type='models.decoders.resnet_add', xyz_condition=True, z_dim=128)), trainer=Namespace(epochs=2000, opt_criticnet=Namespace(beta1=0.9, beta2=0.999, lr='1e-3', momentum=0.9, scheduler='linear', step_epoch=2000, type='adam', weight_decay=0.0), opt_scorenet=Namespace(beta1=0.9, beta2=0.999, lr='1e-3', momentum=0.9, scheduler='linear', step_epoch=2000, type='adam', weight_decay=0.0), seed=100, sigma_begin=1, sigma_end=0.01, sigma_num=10, type='trainers.ae_tr

In [6]:
cfg.data

Namespace(batch_size=32, cates=['airplane'], data_dir='data/ShapeNetCore.v2.PC15k', dataset_scale=1, dataset_type='shapenet15k', normalize_per_shape=False, normalize_std_per_axis=False, num_workers=4, recenter_per_shape=True, te_max_sample_points=2048, tr_max_sample_points=2048, type='datasets.pointflow_datasets')

In [7]:
cfg.trainer

Namespace(epochs=2000, opt_criticnet=Namespace(beta1=0.9, beta2=0.999, lr='1e-3', momentum=0.9, scheduler='linear', step_epoch=2000, type='adam', weight_decay=0.0), opt_scorenet=Namespace(beta1=0.9, beta2=0.999, lr='1e-3', momentum=0.9, scheduler='linear', step_epoch=2000, type='adam', weight_decay=0.0), seed=100, sigma_begin=1, sigma_end=0.01, sigma_num=10, type='trainers.ae_trainer_3D')

In [8]:
cfg.models

Namespace(scorenet=Namespace(dim=3, hidden_size=256, n_blocks=24, out_dim=3, param_likelihood=False, sigma_condition=True, type='models.decoders.resnet_add', xyz_condition=True, z_dim=128))

In [9]:
cfg.inference

Namespace(num_points=2048, num_steps=10, step_size_ratio=1, weight=1)

In [10]:
cfg.viz

Namespace(log_freq=10, save_freq=100, val_freq=100, viz_freq=5000)

# Train

In [11]:
from scorenet import Scorenet
from critic import Criticnet
from utils import get_opt, approx_jacobian_trace, set_random_seed, exact_jacobian_trace
from data_loader import get_data
import random

In [12]:
set_random_seed(getattr(cfg.trainer, "seed", 666))

# load data
train_data = get_data(cfg.data, 0)
tr_pts = train_data['tr_points'].unsqueeze(0)
te_pts = train_data['te_points'].unsqueeze(0)
tr_pts = tr_pts.to(device)
te_pts = te_pts.to(device)
tr_pts.requires_grad_()
te_pts.requires_grad_()

100%|██████████| 1/1 [00:01<00:00,  1.99s/it]


Total number of data:2832
Min number of points: (train)2048 (test)2048


tensor([[[-0.0832, -0.0008, -0.0106],
         [ 0.2124, -0.1133,  0.3423],
         [ 0.3726, -0.1521, -0.1293],
         ...,
         [-0.0380,  0.0209, -0.6827],
         [ 0.0835, -0.0098, -0.6379],
         [ 0.6067, -0.1337, -0.0316]]], device='cuda:0', requires_grad=True)

In [13]:
# sigmas
if hasattr(cfg.trainer, "sigmas"):
    sigmas = cfg.trainer.sigmas
else:
    sigma_begin = float(cfg.trainer.sigma_begin)
    sigma_end = float(cfg.trainer.sigma_end)
    num_classes = int(cfg.trainer.sigma_num)
    sigmas = np.exp(np.linspace(np.log(sigma_begin), np.log(sigma_end), num_classes))
print("Sigma:, ", sigmas)

Sigma:,  [1.         0.59948425 0.35938137 0.21544347 0.12915497 0.07742637
 0.04641589 0.02782559 0.01668101 0.01      ]


In [14]:
# score net
score_net = Scorenet()
critic_net = Criticnet()
critic_net.to(device)
score_net.to(device)

print(score_net)
print(critic_net)

# optimizer
opt_scorenet, scheduler_scorenet = get_opt(score_net.parameters(), cfg.trainer.opt_scorenet)
opt_criticnet, scheduler_criticnet = get_opt(critic_net.parameters(), cfg.trainer.opt_scorenet)

# training
start_epoch = 0
print("Start epoch: %d End epoch: %d" % (start_epoch, cfg.trainer.epochs))
k_iters = 10
e_iters = 10
for epoch in range(start_epoch, cfg.trainer.epochs):
    score_net.train()
    critic_net.train()
    opt_scorenet.zero_grad()
    opt_criticnet.zero_grad()
    
    labels = torch.randint(0, len(sigmas), (1,), device=tr_pts.device)
    used_sigmas = torch.tensor(np.array(sigmas))[-1].float().view(1, 1).cuda()
    
    perturbed_points = tr_pts + torch.randn_like(tr_pts) * used_sigmas
    
    score_pred = score_net(perturbed_points, used_sigmas)
    critic_output = critic_net(perturbed_points, used_sigmas)
    
    t1 = (score_pred * critic_output).sum(-1)
#     t2 = approx_jacobian_trace(critic_output, perturbed_points)
    t2 = exact_jacobian_trace(critic_output, perturbed_points)
    stein = t1 + t2
    l2_penalty = (critic_output * critic_output).sum(-1).mean()
    loss = stein.mean()
    
    cycle_iter = epoch % (k_iters + e_iters)
    cpu_loss = loss.detach().cpu().item()
    cpu_t1 = t1.mean().detach().cpu().item()
    cpu_t2 = t2.mean().detach().cpu().item()
    if cycle_iter < k_iters:
        (-loss + l2_penalty).backward()
        opt_criticnet.step()
        print("Epoch (critic) %d Loss=%2.5f t1=%2.5f t2=%2.5f" % (epoch, cpu_loss, cpu_t1, cpu_t2))
    else:
        loss.backward()
        opt_scorenet.step()
        print("Epoch (score) %d Loss=%2.5f t1=%2.5f t2=%2.5f" % (epoch, cpu_loss, cpu_t1, cpu_t2))

Scorenet(
  (conv_p): Conv1d(4, 256, kernel_size=(1,), stride=(1,))
  (blocks): ModuleList(
    (0): ResnetBlockConv1d(
      (bn_0): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn_1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc_0): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
      (fc_1): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
      (fc_c): Conv1d(4, 256, kernel_size=(1,), stride=(1,))
      (actvn): ReLU()
    )
    (1): ResnetBlockConv1d(
      (bn_0): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn_1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc_0): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
      (fc_1): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
      (fc_c): Conv1d(4, 256, kernel_size=(1,), stride=(1,))
      (actvn): ReLU()
    )
    (2): ResnetBlockConv1d(
      (bn_0): BatchN

Epoch (critic) 63 Loss=16.13451 t1=7.23345 t2=8.90106
Epoch (critic) 64 Loss=19.63734 t1=8.94100 t2=10.69634
Epoch (critic) 65 Loss=47.16737 t1=8.99096 t2=38.17641
Epoch (critic) 66 Loss=41.10990 t1=10.88998 t2=30.21992
Epoch (critic) 67 Loss=70.01431 t1=12.09841 t2=57.91589
Epoch (critic) 68 Loss=15.29436 t1=13.82187 t2=1.47250
Epoch (critic) 69 Loss=14.89438 t1=13.66047 t2=1.23390
Epoch (score) 70 Loss=14.11214 t1=13.03277 t2=1.07937
Epoch (score) 71 Loss=13.50854 t1=12.02393 t2=1.48461
Epoch (score) 72 Loss=10.90442 t1=9.71315 t2=1.19127
Epoch (score) 73 Loss=7.11056 t1=6.11233 t2=0.99823
Epoch (score) 74 Loss=2.91001 t1=1.89366 t2=1.01636
Epoch (score) 75 Loss=-0.62954 t1=-1.63089 t2=1.00135
Epoch (score) 76 Loss=-2.84753 t1=-3.90047 t2=1.05295
Epoch (score) 77 Loss=-4.30332 t1=-5.39706 t2=1.09374
Epoch (score) 78 Loss=-5.77427 t1=-6.85742 t2=1.08315
Epoch (score) 79 Loss=-6.65411 t1=-8.00768 t2=1.35357
Epoch (critic) 80 Loss=-7.70371 t1=-8.90047 t2=1.19675
Epoch (critic) 81 Loss=-

Epoch (score) 216 Loss=7.54088 t1=-1.11864 t2=8.65952
Epoch (score) 217 Loss=6.91474 t1=-1.67509 t2=8.58983
Epoch (score) 218 Loss=15.64135 t1=-2.43766 t2=18.07901
Epoch (score) 219 Loss=2.88045 t1=-2.75199 t2=5.63244
Epoch (critic) 220 Loss=9.75872 t1=-3.33294 t2=13.09166
Epoch (critic) 221 Loss=19.30830 t1=-2.18922 t2=21.49752
Epoch (critic) 222 Loss=32.97780 t1=-1.73415 t2=34.71194
Epoch (critic) 223 Loss=48.11803 t1=-1.68143 t2=49.79946
Epoch (critic) 224 Loss=38.60541 t1=-1.79477 t2=40.40018
Epoch (critic) 225 Loss=40.48336 t1=-0.43520 t2=40.91856
Epoch (critic) 226 Loss=24.33853 t1=0.98494 t2=23.35360
Epoch (critic) 227 Loss=56.22799 t1=1.35095 t2=54.87703
Epoch (critic) 228 Loss=39.05786 t1=1.41220 t2=37.64566
Epoch (critic) 229 Loss=56.44006 t1=2.25206 t2=54.18800
Epoch (score) 230 Loss=33.33117 t1=3.90467 t2=29.42650
Epoch (score) 231 Loss=44.17001 t1=3.11591 t2=41.05410
Epoch (score) 232 Loss=42.05246 t1=2.89813 t2=39.15432
Epoch (score) 233 Loss=12.07530 t1=2.39629 t2=9.6790

Epoch (critic) 366 Loss=2.81148 t1=0.89079 t2=1.92069
Epoch (critic) 367 Loss=2.48245 t1=0.92259 t2=1.55987
Epoch (critic) 368 Loss=3.00827 t1=0.93541 t2=2.07285
Epoch (critic) 369 Loss=2.98616 t1=0.95991 t2=2.02625
Epoch (score) 370 Loss=2.88414 t1=1.00086 t2=1.88328
Epoch (score) 371 Loss=2.61689 t1=0.89967 t2=1.71722
Epoch (score) 372 Loss=2.70927 t1=0.78209 t2=1.92718
Epoch (score) 373 Loss=2.52414 t1=0.66657 t2=1.85757
Epoch (score) 374 Loss=2.48362 t1=0.55167 t2=1.93196
Epoch (score) 375 Loss=2.54757 t1=0.43070 t2=2.11688
Epoch (score) 376 Loss=2.37770 t1=0.30291 t2=2.07479
Epoch (score) 377 Loss=2.02542 t1=0.18697 t2=1.83845
Epoch (score) 378 Loss=2.03824 t1=0.06587 t2=1.97237
Epoch (score) 379 Loss=2.39509 t1=-0.05842 t2=2.45351
Epoch (critic) 380 Loss=1.81140 t1=-0.16877 t2=1.98017
Epoch (critic) 381 Loss=1.78134 t1=-0.11270 t2=1.89403
Epoch (critic) 382 Loss=2.14005 t1=-0.02965 t2=2.16970
Epoch (critic) 383 Loss=2.23782 t1=0.07366 t2=2.16415
Epoch (critic) 384 Loss=2.64116 t1

Epoch (score) 519 Loss=2.03681 t1=0.09217 t2=1.94464
Epoch (critic) 520 Loss=2.07601 t1=0.07387 t2=2.00213
Epoch (critic) 521 Loss=2.35566 t1=0.08810 t2=2.26756
Epoch (critic) 522 Loss=2.69717 t1=0.09182 t2=2.60536
Epoch (critic) 523 Loss=2.90164 t1=0.09979 t2=2.80184
Epoch (critic) 524 Loss=3.10447 t1=0.11081 t2=2.99366
Epoch (critic) 525 Loss=3.97533 t1=0.11908 t2=3.85625
Epoch (critic) 526 Loss=4.20133 t1=0.13006 t2=4.07127
Epoch (critic) 527 Loss=4.91886 t1=0.13561 t2=4.78325
Epoch (critic) 528 Loss=5.51141 t1=0.13924 t2=5.37217
Epoch (critic) 529 Loss=6.36855 t1=0.14474 t2=6.22381
Epoch (score) 530 Loss=8.40068 t1=0.16140 t2=8.23928
Epoch (score) 531 Loss=8.04774 t1=0.15727 t2=7.89047
Epoch (score) 532 Loss=8.71952 t1=0.15912 t2=8.56040
Epoch (score) 533 Loss=8.36503 t1=0.14755 t2=8.21748
Epoch (score) 534 Loss=7.16243 t1=0.13436 t2=7.02807
Epoch (score) 535 Loss=8.73102 t1=0.13433 t2=8.59669
Epoch (score) 536 Loss=8.14272 t1=0.12350 t2=8.01923
Epoch (score) 537 Loss=8.88515 t1=0.

...

Epoch (score) 1413 Loss=14.71752 t1=-0.08507 t2=14.80259
Epoch (score) 1414 Loss=16.17643 t1=-0.09777 t2=16.27420
Epoch (score) 1415 Loss=20.11597 t1=-0.11914 t2=20.23512
Epoch (score) 1416 Loss=16.51806 t1=-0.11352 t2=16.63157
Epoch (score) 1417 Loss=18.36753 t1=-0.14068 t2=18.50821
Epoch (score) 1418 Loss=13.70660 t1=-0.15540 t2=13.86201
Epoch (score) 1419 Loss=8.95134 t1=-0.16825 t2=9.11958
Epoch (critic) 1420 Loss=15.45474 t1=-0.22380 t2=15.67853
Epoch (critic) 1421 Loss=16.85809 t1=-0.19153 t2=17.04962
Epoch (critic) 1422 Loss=34.76453 t1=-0.16978 t2=34.93431
Epoch (critic) 1423 Loss=43.29589 t1=-0.10346 t2=43.39935
Epoch (critic) 1424 Loss=65.17311 t1=-0.05677 t2=65.22988
Epoch (critic) 1425 Loss=71.08200 t1=0.02393 t2=71.05807
Epoch (critic) 1426 Loss=40.49037 t1=0.02386 t2=40.46651
Epoch (critic) 1427 Loss=56.72800 t1=-0.04037 t2=56.76837
Epoch (critic) 1428 Loss=29.53892 t1=-0.02593 t2=29.56485
Epoch (critic) 1429 Loss=19.76507 t1=-0.10539 t2=19.87046
Epoch (score) 1430 Loss=1

Epoch (score) 1557 Loss=165.17880 t1=-2.57421 t2=167.75302
Epoch (score) 1558 Loss=205.95230 t1=-3.54809 t2=209.50040
Epoch (score) 1559 Loss=121.12344 t1=-1.91115 t2=123.03460
Epoch (critic) 1560 Loss=96.18275 t1=-1.04755 t2=97.23028
Epoch (critic) 1561 Loss=132.73827 t1=-4.04684 t2=136.78510
Epoch (critic) 1562 Loss=128.77968 t1=-4.84303 t2=133.62271
Epoch (critic) 1563 Loss=128.61076 t1=-1.90468 t2=130.51544
Epoch (critic) 1564 Loss=159.62968 t1=-1.94510 t2=161.57478
Epoch (critic) 1565 Loss=179.69327 t1=-1.13822 t2=180.83150
Epoch (critic) 1566 Loss=179.11792 t1=-0.37126 t2=179.48920
Epoch (critic) 1567 Loss=207.10498 t1=-4.43299 t2=211.53796
Epoch (critic) 1568 Loss=287.36755 t1=3.27529 t2=284.09229
Epoch (critic) 1569 Loss=294.99374 t1=-2.87085 t2=297.86456
Epoch (score) 1570 Loss=470.64569 t1=4.49639 t2=466.14935
Epoch (score) 1571 Loss=385.32861 t1=6.00304 t2=379.32550
Epoch (score) 1572 Loss=379.28604 t1=4.60784 t2=374.67816
Epoch (score) 1573 Loss=225.83743 t1=9.73740 t2=216.

Epoch (critic) 1701 Loss=39.14417 t1=1.76148 t2=37.38269
Epoch (critic) 1702 Loss=29.70290 t1=1.96824 t2=27.73466
Epoch (critic) 1703 Loss=35.30958 t1=1.77755 t2=33.53203
Epoch (critic) 1704 Loss=37.15858 t1=1.52357 t2=35.63502
Epoch (critic) 1705 Loss=38.54643 t1=1.30741 t2=37.23902
Epoch (critic) 1706 Loss=37.92504 t1=1.15505 t2=36.76999
Epoch (critic) 1707 Loss=43.03540 t1=0.94561 t2=42.08979
Epoch (critic) 1708 Loss=40.85735 t1=0.62298 t2=40.23437
Epoch (critic) 1709 Loss=48.00927 t1=0.17464 t2=47.83463
Epoch (score) 1710 Loss=58.50204 t1=-0.28315 t2=58.78519
Epoch (score) 1711 Loss=53.09857 t1=-0.26298 t2=53.36155
Epoch (score) 1712 Loss=56.70308 t1=-0.29070 t2=56.99378
Epoch (score) 1713 Loss=58.21258 t1=-0.35852 t2=58.57111
Epoch (score) 1714 Loss=57.01678 t1=-0.40212 t2=57.41891
Epoch (score) 1715 Loss=48.00467 t1=-0.42269 t2=48.42735
Epoch (score) 1716 Loss=59.11109 t1=-0.54871 t2=59.65980
Epoch (score) 1717 Loss=56.30276 t1=-0.63237 t2=56.93513
Epoch (score) 1718 Loss=46.4277

Epoch (critic) 1845 Loss=138.63573 t1=33.95533 t2=104.68039
Epoch (critic) 1846 Loss=175.56635 t1=2.56051 t2=173.00583
Epoch (critic) 1847 Loss=260.09039 t1=9.30434 t2=250.78604
Epoch (critic) 1848 Loss=374.25177 t1=20.70671 t2=353.54510
Epoch (critic) 1849 Loss=64.05381 t1=-0.25047 t2=64.30428
Epoch (score) 1850 Loss=255.77942 t1=-0.02269 t2=255.80211
Epoch (score) 1851 Loss=223.22241 t1=-3.22796 t2=226.45038
Epoch (score) 1852 Loss=240.73865 t1=-8.84654 t2=249.58521
Epoch (score) 1853 Loss=272.49460 t1=-13.19289 t2=285.68750
Epoch (score) 1854 Loss=214.80464 t1=-18.19025 t2=232.99487
Epoch (score) 1855 Loss=187.90256 t1=-22.30330 t2=210.20584
Epoch (score) 1856 Loss=194.48819 t1=-25.57534 t2=220.06354
Epoch (score) 1857 Loss=220.26801 t1=-27.93587 t2=248.20386
Epoch (score) 1858 Loss=242.93845 t1=-31.31683 t2=274.25528
Epoch (score) 1859 Loss=206.43835 t1=-35.85236 t2=242.29070
Epoch (critic) 1860 Loss=221.53857 t1=-38.99258 t2=260.53116
Epoch (critic) 1861 Loss=27.02805 t1=20.09760 

Epoch (score) 1995 Loss=5.41733 t1=0.08863 t2=5.32869
Epoch (score) 1996 Loss=4.60465 t1=0.08995 t2=4.51470
Epoch (score) 1997 Loss=5.92712 t1=0.08464 t2=5.84248
Epoch (score) 1998 Loss=4.68067 t1=0.06642 t2=4.61425
Epoch (score) 1999 Loss=4.72752 t1=0.05606 t2=4.67146
